# Dataset Cleaning
Here some approaches to cleaning a dataset will be applied and the result written to file for inspection.

In [1]:
import pandas as pd
import numpy as np

## 1. Read the dataset into a Pandas DataFrame
Pandas DataFrames offer a lot of functionality for inspecting and efficiently manipulating data.

In [2]:
raw_data = "./dataset/interview_signup.csv"
#Note: A converter is necessary when reading in the postcode data in order to ensure 
#      that any leading zeroes are preserved.
df = pd.read_csv(raw_data, converters={'postcode': lambda x: str(x)})
df

,original_product_name,postcode,bundesland,total_bonus,order_date
0,E.ON STROM,53229,Nordrhein-Westfalen,146.0,2018-01-01
1,E.ON STROM ÖKO,74235.0,Baden-Württemberg,67.0,2018-01-01
2,E.ON STROM ÖKO 24,45257,Nordrhein-Westfalen,123.0,2018-01-01
3,E.ON STROM,64395,Hessen,159.0,2018-01-01
4,E.ON STROM 24,36039,Hessen,116.0,2018-01-01
...,...,...,...,...,...
318340,E.ON STROM 24,49610,Niedersachsen,147.0,2018-12-31
318341,E.ON STROM,4916.0,NaN,142.0,2018-12-31
318342,E.ON STROM,77972.0,Baden-Württemberg,48.0,2018-12-31
318343,E.ON STROM ÖKO 24,91466,Bayern,238.0,2018-12-31


## 2. Clean feature: postcode
First any '.0' will be removed from the postcode data.

In [3]:
df['postcode'] = df['postcode'].astype(str).replace('\.0', '', regex=True)
df

,original_product_name,postcode,bundesland,total_bonus,order_date
0,E.ON STROM,53229,Nordrhein-Westfalen,146.0,2018-01-01
1,E.ON STROM ÖKO,74235,Baden-Württemberg,67.0,2018-01-01
2,E.ON STROM ÖKO 24,45257,Nordrhein-Westfalen,123.0,2018-01-01
3,E.ON STROM,64395,Hessen,159.0,2018-01-01
4,E.ON STROM 24,36039,Hessen,116.0,2018-01-01
...,...,...,...,...,...
318340,E.ON STROM 24,49610,Niedersachsen,147.0,2018-12-31
318341,E.ON STROM,4916,NaN,142.0,2018-12-31
318342,E.ON STROM,77972,Baden-Württemberg,48.0,2018-12-31
318343,E.ON STROM ÖKO 24,91466,Bayern,238.0,2018-12-31


Make use of the zfill() helper method to left-pad any string, integer or float with zeros

In [4]:
df['postcode']=[x.zfill(5) for x in df['postcode']]
df

,original_product_name,postcode,bundesland,total_bonus,order_date
0,E.ON STROM,53229,Nordrhein-Westfalen,146.0,2018-01-01
1,E.ON STROM ÖKO,74235,Baden-Württemberg,67.0,2018-01-01
2,E.ON STROM ÖKO 24,45257,Nordrhein-Westfalen,123.0,2018-01-01
3,E.ON STROM,64395,Hessen,159.0,2018-01-01
4,E.ON STROM 24,36039,Hessen,116.0,2018-01-01
...,...,...,...,...,...
318340,E.ON STROM 24,49610,Niedersachsen,147.0,2018-12-31
318341,E.ON STROM,04916,NaN,142.0,2018-12-31
318342,E.ON STROM,77972,Baden-Württemberg,48.0,2018-12-31
318343,E.ON STROM ÖKO 24,91466,Bayern,238.0,2018-12-31


Extract any remaining invalid postcode data for further inspection:

In [5]:
# import a helper for working with PLZ data
import utility.plz_helper as plz_helper

#Create a mask by applying list comprehension to check the validity of each PLZ in the postcode column
mask = [plz_helper.check_invalid_plz_format(str(x)) for x in df['postcode']]
result_df = df[mask]
result_df

,original_product_name,postcode,bundesland,total_bonus,order_date
266922,E.ON STROM,92696JAVAS,NaN,97.0,2018-11-03


Just need to remove alphabetic characters.

In [6]:
import re

df['postcode']=[re.sub("[^0-9]", "", x) for x in df['postcode']]

Check again...

In [7]:
#Create a mask by applying list comprehension to check the validity of each PLZ in the postcode column
mask = [plz_helper.check_invalid_plz_format(str(x)) for x in df['postcode']]
result_df = df[mask]
result_df

,original_product_name,postcode,bundesland,total_bonus,order_date


Hurrah!! Empty DataFrame, hence nothing more to do in terms of ensuring that the postcode data is valid.

## 3. Missing values: bundesland
In order to tackle the missing values for the **bundesland** feature the quickest fix would be to map the PLZ in the postcode column to the required value, i.e. Bundesland.

For that we would require a list of PLZ data with Bundesland. Luckily you can find this on the internet!

Here we will use a csv file obtained from this gist on github: https://gist.github.com/jbspeakr/4565964

It's been forked and some data had to be corrected (Schleswig-Holstein had been mispelled) and missing PLZ data added. 

The forked and updated version used to clean the data can be found here: https://gist.github.com/remuant/7c8f759ae4581e0bb24c6f83808d29fb. It has also been included in this respository in the **resources** folder.

In [8]:
plz_csv_path = './resources/German-Zip-Codes.csv'
plz_data = pd.read_csv(plz_csv_path, sep=';', converters={'Plz': lambda x: str(x)})

In [9]:
plz_data

,Ort,Zusatz,Plz,Vorwahl,Bundesland
0,Aach,b Trier,54298,651.0,Rheinland-Pfalz
1,Aach,", Hegau",78267,7774.0,Baden-Württemberg
2,Aachen,NaN,52062,241.0,Nordrhein-Westfalen
3,Aachen,NaN,52064,241.0,Nordrhein-Westfalen
4,Aachen,NaN,52066,241.0,Nordrhein-Westfalen
...,...,...,...,...,...
19672,Zwingenberg,", Baden",69439,6263.0,Baden-Württemberg
19673,Zwischendeich,NaN,19322,3877.0,Brandenburg
19674,Zwochau,b Delitzsch,04509,34207.0,Sachsen
19675,Zwönitz,NaN,08297,37754.0,Sachsen


Let's check that every Bundesland is included:

In [10]:
plz_data['Bundesland'].unique()

array(['Rheinland-Pfalz', 'Baden-Württemberg', 'Nordrhein-Westfalen',
       'Hessen', 'Schleswig-Holstein', 'Sachsen-Anhalt', 'Brandenburg',
       'Bayern', 'Sachsen', 'Thüringen', 'Niedersachsen',
       'Mecklenburg-Vorpommern', 'Saarland', 'Berlin', 'Bremen',
       'Hamburg'], dtype=object)

We are going to create a dictionary using the Plz and Bundesland columns.

The **PLZ data will form the key** and the **Bundesland will be the value**.

The dictionary can then be used to map PLZ data to a Bundesland

In [11]:
my_plz_dict = dict(zip(plz_data.Plz, plz_data.Bundesland))
#my_plz_dict

Now update the **bundesland** feature with the mapping from the corresponding postcode value (PLZ) to Bundesland if the current value is null:

In [12]:
df['bundesland']=[my_plz_dict[y] if pd.isnull(x) else x for x, y in zip(df['bundesland'], df['postcode'])]
df

,original_product_name,postcode,bundesland,total_bonus,order_date
0,E.ON STROM,53229,Nordrhein-Westfalen,146.0,2018-01-01
1,E.ON STROM ÖKO,74235,Baden-Württemberg,67.0,2018-01-01
2,E.ON STROM ÖKO 24,45257,Nordrhein-Westfalen,123.0,2018-01-01
3,E.ON STROM,64395,Hessen,159.0,2018-01-01
4,E.ON STROM 24,36039,Hessen,116.0,2018-01-01
...,...,...,...,...,...
318340,E.ON STROM 24,49610,Niedersachsen,147.0,2018-12-31
318341,E.ON STROM,04916,Brandenburg,142.0,2018-12-31
318342,E.ON STROM,77972,Baden-Württemberg,48.0,2018-12-31
318343,E.ON STROM ÖKO 24,91466,Bayern,238.0,2018-12-31


Are there still missing values?

In [13]:
df.isnull().sum()

original_product_name    0
postcode                 0
bundesland               0
total_bonus              0
order_date               0
dtype: int64

All missing **bundesland** feature values have been updated with the correct value as determined from the postcode feature value.

## 4.  Correct typos: original_product_name
The approach presented here is quick but limited to the errors in the data which were observed.
It has been included to round off the processing of the dataset.

A better more reusable approach would be to match the feature values to the best match from a set of tarif names based on a measure of similarity/distance, e.g. Levenshtein distance.

The approach taken here is to construct a dictionary of valid tarif names (keys) and the values which should map to them (values). The values are taken from the observed feature names found in the raw dataset.

Now the dictionary is created and checked:

In [14]:
tarif_dict = dict()
tarif_dict['E.ON STROM']=('E.ON STROM')
tarif_dict['E.ON STROM ÖKO']=('E.ON STROM ÖKO','E.ON STROM Ã–KO','E.ON STROM ÖO')
tarif_dict['E.ON STROM ÖKO 24']=('E.ON STROM ÖKO 24')
tarif_dict['E.ON STROM 24']=('E.ON STROM 24','E.ON STROM 24 24 24','E.ON STROM 24 24','E.ON STROM 24 24 24 24 24 24 24')
tarif_dict['E.ON STROM PUR']=('E.ON STROM PUR')
tarif_dict

{'E.ON STROM': 'E.ON STROM',
 'E.ON STROM ÖKO': ('E.ON STROM ÖKO', 'E.ON STROM Ã–KO', 'E.ON STROM ÖO'),
 'E.ON STROM ÖKO 24': 'E.ON STROM ÖKO 24',
 'E.ON STROM 24': ('E.ON STROM 24',
  'E.ON STROM 24 24 24',
  'E.ON STROM 24 24',
  'E.ON STROM 24 24 24 24 24 24 24'),
 'E.ON STROM PUR': 'E.ON STROM PUR'}

In [15]:
tarif_dict.items()

dict_items([('E.ON STROM', 'E.ON STROM'), ('E.ON STROM ÖKO', ('E.ON STROM ÖKO', 'E.ON STROM Ã–KO', 'E.ON STROM ÖO')), ('E.ON STROM ÖKO 24', 'E.ON STROM ÖKO 24'), ('E.ON STROM 24', ('E.ON STROM 24', 'E.ON STROM 24 24 24', 'E.ON STROM 24 24', 'E.ON STROM 24 24 24 24 24 24 24')), ('E.ON STROM PUR', 'E.ON STROM PUR')])

A function has been defined in a helper which gets the key (valid tarif name) by accessing the associated values, import the helper:

In [16]:
import utility.misc_helper as mh

Now the mapping is performed for all values of the feature:

In [17]:
df['original_product_name'] = [mh.get_key_from_value(x,tarif_dict) for x in df['original_product_name']]

In [18]:
df

,original_product_name,postcode,bundesland,total_bonus,order_date
0,E.ON STROM,53229,Nordrhein-Westfalen,146.0,2018-01-01
1,E.ON STROM ÖKO,74235,Baden-Württemberg,67.0,2018-01-01
2,E.ON STROM ÖKO 24,45257,Nordrhein-Westfalen,123.0,2018-01-01
3,E.ON STROM,64395,Hessen,159.0,2018-01-01
4,E.ON STROM 24,36039,Hessen,116.0,2018-01-01
...,...,...,...,...,...
318340,E.ON STROM 24,49610,Niedersachsen,147.0,2018-12-31
318341,E.ON STROM,04916,Brandenburg,142.0,2018-12-31
318342,E.ON STROM,77972,Baden-Württemberg,48.0,2018-12-31
318343,E.ON STROM ÖKO 24,91466,Bayern,238.0,2018-12-31


Now check whether the feature values comprise only valid tarifs:

In [19]:
df['original_product_name'].unique()

array(['E.ON STROM', 'E.ON STROM ÖKO', 'E.ON STROM ÖKO 24',
       'E.ON STROM 24', 'E.ON STROM PUR'], dtype=object)

All valid! Cleaning is complete.

## Write the cleaned dataset to file
Store the resulting dataset as a csv in the outputs folder.

In [20]:
output_path = './output/cleaned_customer_data__jupyter.csv'
df.to_csv(output_path, index = False)